In [48]:
import os 
import pandas as pd


In [53]:
vector_data=pd.read_csv('D:\Model_Deployement\End_to_End_Phishing_mail_detection_ML_project\\artifacts\data_preprocessed\\vectorized_data.csv',index_col=False)

In [54]:
k=vector_data['Email_vector']

In [55]:
import numpy as np
def convert_string_to_array(vector_string):
    """Convert string representation of vector to numpy array"""
    try:
        # Remove brackets and split by whitespace
        vector_string = vector_string.strip('[]')
        # Convert to numpy array
        return np.fromstring(vector_string, sep=' ')
    except:
        return None

vector_data['Email_vector'] = vector_data['Email_vector'].apply(convert_string_to_array)

    
 # Example usage, convert the first vector string to numpy array

In [61]:
vector_data[vector_data.columns[2:]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18650 entries, 0 to 18649
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Email Text    18634 non-null  object
 1   Email Type    18650 non-null  int64 
 2   tokens        18650 non-null  object
 3   Email_vector  18650 non-null  object
dtypes: int64(1), object(3)
memory usage: 582.9+ KB


In [23]:
vec=np.vstack(new)

In [25]:
vec.shape

(100, 1)

In [62]:
vector_data['Email Type'].replace({'Safe Email':0,'Phishing Email':1},inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_12088\2393261195.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  vector_data['Email Type'].replace({'Safe Email':0,'Phishing Email':1},inplace=True)


In [63]:
vector_data['Email Type']

0        0
1        0
2        0
3        1
4        1
        ..
18645    1
18646    0
18647    0
18648    0
18649    1
Name: Email Type, Length: 18650, dtype: int64

In [65]:
vector_data['Email Type'].unique()

array([0, 1], dtype=int64)

# Model training on the data

In [ ]:
import os
from pathlib import Path


In [47]:
os.getcwd()

'd:\\Model_Deployement\\End_to_End_Phishing_mail_detection_ML_project'

In [46]:
os.chdir('../')

In [ ]:
from dataclasses import dataclass
@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir :Path
    training_data_path: Path
    trained_model_dir: Path
